In [1]:
import numpy as np
import pandas as pd
from sympy import diff, lambdify, symbols, log, sqrt
from sympy.stats import Normal, density
from scipy.stats import norm

kappa = 0.1465
alpha = 0.5172
omega = 0.5786
rho = -0.0243
s0 = 1000
r = 0
T = 1 / 12
K = 1000
xi = 0.5
# symbolic variables
s, v, t, v0 = symbols('s v t v0')
N = Normal('x', 0, 1)
d_p = (log(s / K) + (r + v0 / 2) * t) / sqrt(v0 * t)
delta0 = 0.5 * (v - v0) * s * density(N)(d_p) / sqrt(v0 * t)

def bs(S, K, T, r, sigma):
  N = norm.cdf
  d_p = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
  d_m = d_p - sigma * np.sqrt(T)
  return S * N(d_p) - K * np.exp(-r * T) * N(d_m)

def L(f): # infinitesimal generator
  ft = diff(f, t)
  fs = diff(f, s)
  fs2 = diff(fs, s)
  fv = diff(f, v)
  fv2 = diff(fv, v)
  fvs = diff(fv, s)
  tmp1 = -ft + r * s * fs + kappa * (alpha - v) * fv
  tmp2 = (v * s**2 * fs2 + omega**2 * v**(2 * xi) * fv2) / 2
  tmp3 = rho * omega * v**(xi + 0.5) * s * fvs
  return tmp1 + tmp2 + tmp3 - r * f

delta = [delta0]
delta_func = []
for _ in range(1, 5):
  delta.append(L(delta[-1]))
for i in range(0, 5):
  delta_func.append(lambdify([s, v, t, v0], delta[i]))

V = np.arange(0.1, 1.2, 0.1)
Approx_4 = []
for v in V:
  p0_approx = bs(s0, K, T, r, np.sqrt(v)) + T * delta_func[0](s0, v, T, v)
  p1_approx = p0_approx + T**2 / 2 * delta_func[1](s0, v, T, v)
  p2_approx = p1_approx + T**3 / 6 * delta_func[2](s0, v, T, v)
  p3_approx = p2_approx + T**4 / 24 * delta_func[3](s0, v, T, v)
  p4_approx = p3_approx + T**5 / 120 * delta_func[4](s0, v, T, v)
  Approx_4.append(p4_approx)

FFT = np.array([36.4488, 51.4125, 62.8997, 72.5792, 81.1007, 88.7981, 95.8702, 102.4465, 108.6171, 114.4477, 119.9878])
JFE_results = np.array([36.4854, 51.4255, 62.9068, 72.5838, 81.1040, 88.8006, 95.8721, 102.4481, 108.6184, 114.4488, 119.9888])
JFE_results_diff = np.array([0.1004, 0.0253, 0.0113, 0.0063, 0.0041, 0.0028, 0.0021, 0.0016, 0.0013, 0.0010, 0.0008])

df = pd.DataFrame({"V": V, "FFT": FFT, "Replicate price": np.round(Approx_4, 4), "JFE price": JFE_results, "JFE %diff": JFE_results_diff})
print(df)
df.to_csv("Table 1 Panel B.csv")

      V       FFT  Replicate price  JFE price  JFE %diff
0   0.1   36.4488          36.4854    36.4854     0.1004
1   0.2   51.4125          51.4255    51.4255     0.0253
2   0.3   62.8997          62.9068    62.9068     0.0113
3   0.4   72.5792          72.5838    72.5838     0.0063
4   0.5   81.1007          81.1040    81.1040     0.0041
5   0.6   88.7981          88.8006    88.8006     0.0028
6   0.7   95.8702          95.8721    95.8721     0.0021
7   0.8  102.4465         102.4481   102.4481     0.0016
8   0.9  108.6171         108.6184   108.6184     0.0013
9   1.0  114.4477         114.4488   114.4488     0.0010
10  1.1  119.9878         119.9888   119.9888     0.0008
